In [1]:
# import os
# os.environ["NUMBA_DISABLE_JIT"] = "1"
# os.environ["NUMBA_DEBUGINFO"] = "1"
# os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

import quantfreedom as qf
import numpy as np
import pandas as pd
import talib
from itertools import product
from talib import abstract
from quantfreedom.backtester.enums.enums import *
from quantfreedom.backtester.base.base import *

np.set_printoptions(precision=4)

In [2]:
# data = qf.CCXTData.data_download(
#     exchange='bybit',
#     start='2020-10-01T00:00:00Z',
#     end='2021-02-23T00:00:00Z',
#     symbol='BTCUSDT',
#     timeframe='30m',
# )
# data.to_csv('30min.csv')
data = pd.read_csv('30min.csv', index_col='Open Time')

In [53]:
timeperiods = np.arange(15, 31, 5).tolist()
temp_rsi = np.empty((data.close.shape[0], len(timeperiods)))
timeperiod_index = pd.Index(timeperiods, name='rsi_timeperiod')

below_ranges = np.arange(20, 51, 5).tolist()
temp_rsi_below = np.empty((data.close.shape[0], (len(timeperiods)*len(below_ranges))), dtype=np.bool_)
# below_index = pd.Index(below_ranges, name='rsi_below')
c = 0
time_zip, below_zip = zip(*product(timeperiods, below_ranges))

In [64]:
for i in range(len(timeperiods)):
    temp_rsi[:, i] = talib.RSI(
        data.close,
        timeperiod=timeperiods[i],
    )
pd.DataFrame(temp_rsi, index=data.close.index, columns=pd.Index(timeperiods, name='rsi_timeperiod'))

rsi_timeperiod,15,20,25,30
time,,,,
2020-10-01 00:00:00,NaN,NaN,NaN,NaN
2020-10-01 00:30:00,NaN,NaN,NaN,NaN
2020-10-01 01:00:00,NaN,NaN,NaN,NaN
2020-10-01 01:30:00,NaN,NaN,NaN,NaN
2020-10-01 02:00:00,NaN,NaN,NaN,NaN
...,...,...,...,...
2021-02-22 21:30:00,55.516071,52.364915,50.642453,49.668702
2021-02-22 22:00:00,51.496913,49.398176,48.236979,47.611461
2021-02-22 22:30:00,49.684656,48.050665,47.140188,46.671107


In [67]:
timeperiods = np.arange(15, 31, 5).tolist()
temp_rsi = np.empty((data.close.shape[0], len(timeperiods)))
# timeperiod_index = pd.Index(timeperiods, name='rsi_timeperiod')

below_ranges = np.arange(20, 51, 5).tolist()
temp_rsi_below = np.empty(
    (data.close.shape[0], (len(timeperiods)*len(below_ranges))), dtype=np.bool_)
# below_index = pd.Index(below_ranges, name='rsi_below')
c = 0
time_zip, below_zip = zip(*product(timeperiods, below_ranges))
for i in range(len(timeperiods)):
    temp_rsi[:, i] = talib.RSI(
        data.close,
        timeperiod=timeperiods[i],
    )
    for x in range(len(below_ranges)):
        temp_rsi_below[:, c] = np.where(
            temp_rsi[:, i] < below_ranges[x], True, False
        )
        c += 1

entries = pd.DataFrame(
    temp_rsi_below,
    index=data.close.index,
    columns=pd.MultiIndex.from_product(
        [below_ranges, timeperiods],
        names=['rsi_below', 'rsi_timeperiod'])
)
entries

rsi_below               20                          25                       \
rsi_timeperiod          15     20     25     30     15     20     25     30   
time                                                                          
2020-10-01 00:00:00  False  False  False  False  False  False  False  False   
2020-10-01 00:30:00  False  False  False  False  False  False  False  False   
2020-10-01 01:00:00  False  False  False  False  False  False  False  False   
2020-10-01 01:30:00  False  False  False  False  False  False  False  False   
2020-10-01 02:00:00  False  False  False  False  False  False  False  False   
...                    ...    ...    ...    ...    ...    ...    ...    ...   
2021-02-22 21:30:00  False  False  False  False  False  False  False  False   
2021-02-22 22:00:00  False  False  False  False  False  False  False  False   
2021-02-22 22:30:00  False  False  False  False  False  False   True  False   
2021-02-22 23:00:00  False  False  False  False  False  False   True  False   
2021-02-22 23:30:00  False  False  False  False  False  False   True  False   

rsi_below               30         ...     40            45                \
rsi_timeperiod          15     20  ...     25     30     15     20     25   
time                               ...                                      
2020-10-01 00:00:00  False  False  ...  False  False  False  False  False   
2020-10-01 00:30:00  False  False  ...  False  False  False  False  False   
2020-10-01 01:00:00  False  False  ...  False  False  False  False  False   
2020-10-01 01:30:00  False  False  ...  False  False  False  False  False   
2020-10-01 02:00:00  False  False  ...  False  False  False  False  False   
...                    ...    ...  ...    ...    ...    ...    ...    ...   
2021-02-22 21:30:00  False  False  ...  False  False  False  False  False   
2021-02-22 22:00:00  False  False  ...  False  False   True  False  False   
2021-02-22 22:30:00  False  False  ...  False  False   True  False  False   
2021-02-22 23:00:00  False  False  ...  False   True   True  False  False   
2021-02-22 23:30:00  False  False  ...  False  False   True  False  False   

rsi_below                      50                       
rsi_timeperiod          30     15     20     25     30  
time                                                    
2020-10-01 00:00:00  False  False  False  False  False  
2020-10-01 00:30:00  False  False  False  False  False  
2020-10-01 01:00:00  False  False  False  False  False  
2020-10-01 01:30:00  False  False  False  False  False  
2020-10-01 02:00:00  False  False  False  False  False  
...                    ...    ...    ...    ...    ...  
2021-02-22 21:30:00  False  False  False  False   True  
2021-02-22 22:00:00  False  False  False  False   True  
2021-02-22 22:30:00  False  False  False  False   True  
2021-02-22 23:00:00  False  False  False   True   True  
2021-02-22 23:30:00  False  False  False  False   True  

[6960 rows x 28 columns]

In [68]:
account_state = AccountAndTradeState(
    available_balance=100.,
    equity=100.,
)
order = OrderEverything(
    lev_mode=np.array([LeverageMode.LeastFreeCashUsed]),
    order_type=np.array([OrderType.LongEntry]),
    size_type=np.array([SizeType.RiskPercentOfAccount]),
    size_pct=np.array([1.]),
    max_equity_risk_pct=np.arange(5, 6.1, 1),
    sl_pcts=np.arange(5, 6.1, .25),
    risk_rewards = np.arange(5, 6.1, .5),
)

In [69]:
df_array, order_records, _ = cart_from_signals(
    open=data.open,
    high=data.high,
    low=data.low,
    close=data.close,
    entries=entries,
    order=order,
    account_state=account_state,
)

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1mnon-precise type pyobject[0m
[0m[1mDuring: typing of argument at e:\coding\backtesters\quantfreedom\quantfreedom\backtester\nb\execute_funcs.py (62)[0m
[1m
File "..\quantfreedom\backtester\nb\execute_funcs.py", line 62:[0m
[1mdef cart_tester(
    <source elided>

[1m    total_order_settings = sl_pcts.size
[0m    [1m^[0m[0m 

This error may have been caused by the following argument(s):
- argument 4: [1mCannot determine Numba type of <class 'pandas.core.frame.DataFrame'>[0m


In [ ]:
df = pd.DataFrame(df_array)
pd.options.display.float_format = '{:,.2f}'.format

# for idx, name in enumerate(entries.columns.names):
#     df[name] = df['ind_set']
#     for jjj in range(df['ind_set'].min(), df['ind_set'].max()+1):
#         df[name].mask(df['ind_set'] == jjj, entries.columns[jjj][idx], inplace=True)
df.sort_values(by=['to_the_upside'], ascending=False).head(30)